# Ridge detection with Complex Shearlets Transform

In [ ]:
import os
import cv2
import math
import numpy as np
from osgeo import gdal
import ipywidgets as widgets
from IPython.display import display
from matplotlib import pyplot as plt
from coshrem.shearletsystem import RidgeSystem
%matplotlib inline

## read georeferencing information if availabe

In [ ]:
georef = False
filename = 'img/TEST.tif'
#filename = 'img/gawler_dem.tif'
#filename = 'img/Ortho_3_061.png'
outfile = 'Test_cs_ref.tif'

dataset = gdal.Open(filename, gdal.GA_ReadOnly)
if dataset:
    print("Driver: {}/{}".format(dataset.GetDriver().ShortName,
                                dataset.GetDriver().LongName))
    print("Size is {} x {} x {}".format(dataset.RasterXSize,
                                        dataset.RasterYSize,
                                        dataset.RasterCount))
    if dataset.GetProjection():
        print("Projection is: {}".format(dataset.GetProjection()))
        geotransform = dataset.GetGeoTransform()
        georef = True
        if geotransform:
            print("Geotransform:" ,geotransform)
            print("Origin = ({}, {})".format(geotransform[0], geotransform[3]))
            print("Pixel Size = ({}, {})".format(geotransform[1], geotransform[5]))
            
            gray = np.array(dataset.GetRasterBand(1).ReadAsArray())
    else:
        image = cv2.imread(filename)    
        gray  = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
'''
scale_percent = 100
width = int(gray.shape[1] * scale_percent / 100)
height = int(gray.shape[0] * scale_percent / 100)
dim = (width, height)
gray = cv2.resize(gray, dim, interpolation = cv2.INTER_AREA)   
'''
MASTER = np.zeros(gray.shape, np.double)
p = gray.shape

## waveletEffSupp
Length of the effective support in pixels of the Mexican hat wavelet
ψ used in the construction the generating shearlet ψgen(x, y) =
ψ(x)φ(y), where φ is a Gaussian. The effective support is the interval
on which the values of ψ significantly differ from 0. It is, however,
not a strictly defined property. A good choice for this parameter is
often 1/8 of the image width. If the edges/ridges in the processed
image are visible on a large scale, this value should be large relative
to the width and height of the processed image.

## gaussianEffSupp
Length of the effective support in pixels of the Gaussian φ used in the
construction of the generating shearlet ψgen(x, y) = ψ(x)φ(y), where
ψ is a Mexican hat wavelet. Typically, this value is chosen to be
roughly the half of waveletEffSupp. However, if the edges/ridges
in the processed image consist of smooth curves, it can be chosen
larger.

## scalesPerOctave
Determines the number of intermediate scales for each octave. If
scalesPerOctave is set to n, for each orientation, there will be n
differently scaled shearlets within one octave.

## shearLevel (orientations)
Determines the number of differently oriented shearlets on each scale.
If shearLevel is set to n, there will be 2n + 2 differently sheared
shearlets on each scale, completing a 180◦ semi-circle.

## alpha (orientations)
This parameter can take any value between 0 and 1 and governs
the degree of anisotropy introduced via scaling. Roughly speaking,
it determines how much the Gaussian is squeezed relative to the
wavelet, when scaling the generating shearlet. Formally, the n-th
octave is defined by ψn(x, y) = ψgen(2nx, 2αny). For alpha = 0, the
degree of anisotropy is maximized while for alpha = 1, both directions
are treated the same.

## octaves
The number of octaves spanned by the shearlet system. When scales-
PerOctave is greater than 1, this parameter can also take non-integer
values.

## minContrast
Specifies the minimal contrast for an edge/ridge to be detected.

## offset
This parameter defines a scaling offset between the even- and odd-
symmetric shearlets measured in octaves. If offset = x, the first
even-symmetric shearlet used for the computation of the complex
shearlet-based edge measure is already x octaves above the first odd-
symmetric shearlet considered. In the case of the ridge measure, the
converse is true.

## scalesUsedForPivotSearch
This parameter defines which scales of the shearlet system are
considered for determining the orientation for which the complex
shearlet-based edge/ridge measure is computed at a specific loca-
tion. It can take the values ’all’, ’highest’, ’lowest’ and any subset
B ⊂ {1, . . . , scalesPerOctave·octaves}.

In [ ]:
with open('img/CS6l6tB.gif', 'rb') as f:
    img = f.read()
loading_bar = widgets.Image(value=img)
out = widgets.Output()
display(out)
def CSridges(p, wavelet_eff_supp = 60, gaussian_eff_supp = 30, scales_per_octave = 4, shear_level = 3, alpha = 0.2, octaves = 3.5,
                          min_contrast = 10, offset = 1, pivoting_scales = 'all', positive = False, negative = True):
    
    sys = RidgeSystem(*p,
                wavelet_eff_supp = wavelet_eff_supp,
                gaussian_eff_supp = gaussian_eff_supp,
                scales_per_octave = scales_per_octave,
                shear_level = shear_level,
                alpha = alpha,
                octaves = octaves)      
    
    features, orientations = sys.detect(gray, 
                                    min_contrast = min_contrast,  
                                    offset = offset, 
                                    pivoting_scales= pivoting_scales,
                                    negative_only = negative,
                                    positive_only = positive )
    return(features)

def InteracRidges(wavelet_eff_supp = 60, gaussian_eff_supp = 30, scales_per_octave = 4, shear_level = 3, alpha = 0.2, octaves = 3.5,
                  min_contrast = 10, offset = 1, pivoting_scales = 'all', positive = False, negative = True): 
    
    f, ax1 = plt.subplots(nrows=1,figsize=(25,25))
    with out:
        display(loading_bar)
        ridges = CSridges(p, wavelet_eff_supp = wavelet_eff_supp, gaussian_eff_supp = gaussian_eff_supp, scales_per_octave = scales_per_octave, shear_level = shear_level, alpha = alpha,
                     min_contrast = min_contrast, offset = offset, pivoting_scales = pivoting_scales, positive = positive, negative = negative)
        out.clear_output()
   
    x1, x2 = ridges.shape[:2] 
    MASTER[:x1, :x2] = ridges[:x1, :x2]

    overlay = cv2.addWeighted(gray,0.001, MASTER ,0.99,0, dtype=cv2.CV_64F)       
    ax1.imshow(overlay, cmap="gray"); 
    ax1.get_xaxis().set_visible(False)
    ax1.axes.get_yaxis().set_visible(False)
    Msum = np.sum(MASTER)
    has_nan = np.isnan(Msum)
    if (has_nan):
        print("NaN in array, check parameter combination.")

style = {'description_width': 'initial'}
widgets.interactive(InteracRidges, wavelet_eff_supp = widgets.IntSlider(description='waveletEffSupp',style=style, value=70 ,min=1, max=1000, step=1,continuous_update=False, layout=widgets.Layout(width='50%')),
                                     gaussian_eff_supp = widgets.IntSlider(description='waveletEffSupp',style=style, value=25 ,min=1, max=1000, step=1,continuous_update=False, layout=widgets.Layout(width='50%')),
                                       scales_per_octave = widgets.IntSlider(description='scalesPerOctave',style=style, value=2, min=1, max=100, step=1,continuous_update=False, layout=widgets.Layout(width='50%')),
                                       shear_level = widgets.IntSlider(description='shearLevel',style=style, value=3, min=1, max=10, step=1,continuous_update=False, layout=widgets.Layout(width='50%')), 
                                       alpha = widgets.FloatSlider(description='alpha',style=style, value=0.5, min=0, max=10, step=0.1,continuous_update=False, layout=widgets.Layout(width='50%')), 
                                       octaves = widgets.FloatSlider(description='octaves',style=style, value=3.5, min=0.0, max=10, step=0.1,continuous_update=False, layout=widgets.Layout(width='50%')),
                                       min_contrast = widgets.IntSlider(description='minContrast',style=style, value=4, min=1, max=100, step=1,continuous_update=False, layout=widgets.Layout(width='50%')),  
                                       offset = widgets.FloatSlider(description='offset',style=style, value=1, min=1, max=100, step=0.1,continuous_update=False, layout=widgets.Layout(width='50%')),
                                       pivoting_scales = widgets.Dropdown(description='scalesUsedForPivotSearch',style=style, options=['all', 'highest', 'lowest'], value='all', layout=widgets.Layout(width='50%'))
                                       )   

## write final image to file

In [ ]:
if georef:
    driver = gdal.GetDriverByName("GTiff")
    outdata = driver.Create(outfile, dataset.RasterXSize, dataset.RasterYSize, 1, gdal.GDT_Float32)
    outdata.SetGeoTransform(dataset.GetGeoTransform())
    outdata.SetProjection(dataset.GetProjection())
    outdata.GetRasterBand(1).WriteArray(MASTER)
    outdata.FlushCache() 
else:
    cv2.imwrite(outfile, MASTER)

In [ ]:
dataset = None